In [83]:
import mne
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

In [84]:
import pandas as pd

# Training
unfiltered_subjects = ['Aanand_19_01.csv','subhojit_24_02.csv','Abhijeet_6205876348.csv','Nishkarsh_22_01.csv','Aanand_19_02.csv',
                       'Deepanshu_21_1.csv',  'Deepanshu_21_02.csv', 'Mantavya_20_01.csv', 'subhojit_24_01.csv',  'Adit_18_02.csv',
                       'Mantavya_20_02.csv','Dipanjan_21_01.csv', 'Pritesh_22_01.csv', 'Rishab_20_02.csv', 'mohitt.csv',
                       'Rishab_20_01.csv', 'Ritwik_23_02.csv', 'Dipanjan_21_02.csv', 'Nishkarsh_22_02.csv','Pritesh_22_02.csv']
sfreq = 120
ch_names = ['timestamps','Fp1','Fp2','C3','C4','P7','P8','O1','O2','F7','F8','F3','F4',
            'T7','T8','P3','P4','stim','sfreq']

# Create an empty DataFrame to store all training data
all_data = pd.DataFrame(columns=ch_names)

for file_name in unfiltered_subjects:
    df = pd.read_csv(file_name)  # Specify index column
    df['timestamps'] = df['timestamps'] - df['timestamps'][0]
    
    lvl1_sub = df[(df['timestamps'] >= 0) & (df['timestamps'] < 150)].copy()
    lvl2_sub = df[(df['timestamps'] >= 150) & (df['timestamps'] < 280)].copy()
    lvl3_sub = df[(df['timestamps'] >= 280) & (df['timestamps'] < 400)].copy()

    lvl1_sub['label'] = 1
    lvl2_sub['label'] = 2
    lvl3_sub['label'] = 3
    
    combined_df = pd.concat([lvl1_sub, lvl2_sub, lvl3_sub])
    all_data = pd.concat([all_data, combined_df])

# Testing
testing_subjects = ['Achal_19_01.csv','Ritwik_23_01.csv']
testing_data = pd.DataFrame(columns=ch_names)

for file_name in testing_subjects:
    df = pd.read_csv(file_name)  # Specify index column
    df['timestamps'] = df['timestamps'] - df['timestamps'][0]
    
    lvl1_sub = df[(df['timestamps'] >= 0) & (df['timestamps'] < 150)].copy()
    lvl2_sub = df[(df['timestamps'] >= 150) & (df['timestamps'] < 280)].copy()
    lvl3_sub = df[(df['timestamps'] >= 280) & (df['timestamps'] < 400)].copy()

    lvl1_sub['label'] = 1
    lvl2_sub['label'] = 2
    lvl3_sub['label'] = 3
    
    combined_df = pd.concat([lvl1_sub, lvl2_sub, lvl3_sub])
    testing_data = pd.concat([testing_data, combined_df])


C:\Users\LEGION\AppData\Local\Temp\ipykernel_19020\3091360791.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_data = pd.concat([all_data, combined_df])
C:\Users\LEGION\AppData\Local\Temp\ipykernel_19020\3091360791.py:47: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  testing_data = pd.concat([testing_data, combined_df])


In [85]:
drop1 = all_data.drop(['sfreq'], axis=1)
df1 = drop1.drop(['stim'], axis=1)
df = df1.drop(['label'], axis=1)

drop_data = testing_data.drop(['sfreq'], axis=1)
df2 = drop_data.drop(['stim'], axis=1)
testing = df2.drop(['label'], axis=1)

In [86]:
eeg_data = df.iloc[:,1:18].values.T
eeg_data[:-1] = eeg_data[:-1] * (10*(-6))
eeg_data[:-1] = eeg_data[:-1] / 4

ch_names = ['Fp1',	'Fp2'	,'C3'	,'C4'	,'P7'	,'P8'	,'O1'	,'O2'	,'F7'	,'F8'	,'F3'	,'F4'	,'T7'	,'T8'	,'P3'	,'P4']
info = mne.create_info(ch_names = ch_names, sfreq = sfreq, ch_types = ( ["eeg" ]*16))
raw = mne.io.RawArray(eeg_data, info)
raw.set_eeg_reference()

raw.filter(l_freq = 1,h_freq = 30)

Creating RawArray with float64 data, n_channels=16, n_times=985812
    Range : 0 ... 985811 =      0.000 ...  8215.092 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 397 samples (3.308 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.6s finished


Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,Not available
Good channels,16 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,120.00 Hz
Highpass,1.00 Hz
Lowpass,30.00 Hz


In [87]:
import mne

# EEG data preprocessing for testing
testing_eeg_data = testing.iloc[:, 1:18].values.T  # Selecting EEG channels only
testing_eeg_data[:-1] = testing_eeg_data[:-1] * (10 ** (-6))  # Convert to microvolts
testing_eeg_data[:-1] = testing_eeg_data[:-1] / 4  # Divide by 4

# Create channel names
ch_names = ['Fp1', 'Fp2', 'C3', 'C4', 'P7', 'P8', 'O1', 'O2', 'F7', 'F8', 'F3', 'F4', 'T7', 'T8', 'P3', 'P4']

info_testing = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=["eeg"] * 16)
raw_testing = mne.io.RawArray(testing_eeg_data, info_testing)
raw_testing.set_eeg_reference()
raw_testing.filter(l_freq=1, h_freq=30)


Creating RawArray with float64 data, n_channels=16, n_times=93143
    Range : 0 ... 93142 =      0.000 ...   776.183 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 397 samples (3.308 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.1s finished


Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,Not available
Good channels,16 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,120.00 Hz
Highpass,1.00 Hz
Lowpass,30.00 Hz


In [88]:
duration = 0.4
epochs=mne.make_fixed_length_epochs(raw, duration = duration, overlap=0,preload=True)

Not setting metadata
20537 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 20537 events and 48 original time points ...
0 bad epochs dropped


In [89]:
# Define epoch duration
duration = 0.4

# Create fixed-length epochs for testing data
testing_epochs = mne.make_fixed_length_epochs(raw_testing, duration=duration, overlap=0, preload=True)


Not setting metadata
1940 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1940 events and 48 original time points ...
0 bad epochs dropped


In [90]:
epochs.drop_bad()  # This will drop bad epochs
num_epochs = len(epochs)
print("Total number of epochs created:", num_epochs)

Total number of epochs created: 20537


In [91]:
testing_epochs.drop_bad()  # This will drop bad epochs
testing_num_epochs = len(testing_epochs)
print("Total number of epochs created for testing:", testing_num_epochs)

Total number of epochs created for testing: 1940


In [92]:
import mne

eeg_data = epochs.get_data()

condition_ranges = {'lev1': (0, (num_epochs/6)),
                    'lev2': (3*(num_epochs/6) , 4*((num_epochs)/6) ),
                    'lev3': ( 5*((num_epochs)/6) , (num_epochs) )}

epochs_by_condition = {}
selected_epochs = []

for condition, (start_epoch, end_epoch) in condition_ranges.items():
    start_epoch = int(start_epoch)
    end_epoch = int(end_epoch)
    condition_epochs = epochs[start_epoch:end_epoch+1].copy()
    epochs_by_condition[condition] = condition_epochs
    selected_epochs.append(condition_epochs)

selected_epochs = mne.concatenate_epochs(selected_epochs)

for condition, epoch_data in epochs_by_condition.items():
    num_epochs = len(epoch_data)
    print(f"Number of epochs for {condition}: {num_epochs}")

print(f"Total number of selected epochs: {len(selected_epochs)}")


C:\Users\LEGION\AppData\Local\Temp\ipykernel_19020\1210157350.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  eeg_data = epochs.get_data()


Not setting metadata
10270 matching events found
No baseline correction applied
Number of epochs for lev1: 3423
Number of epochs for lev2: 3424
Number of epochs for lev3: 3423
Total number of selected epochs: 10270


In [93]:

condition_of_interest = 'lev1'
epoch_array_for_condition = epochs_by_condition[condition_of_interest]
print(f"Epoch array shape for condition {condition_of_interest}: {epoch_array_for_condition.get_data().shape}")


Epoch array shape for condition lev1: (3423, 16, 48)


C:\Users\LEGION\AppData\Local\Temp\ipykernel_19020\1489624503.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  print(f"Epoch array shape for condition {condition_of_interest}: {epoch_array_for_condition.get_data().shape}")


In [94]:
# Define condition ranges based on the number of epochs in the testing data
num_epochs = len(testing_epochs)
condition_ranges_testing = {'lev1': (0, (num_epochs/6)),
                    'lev2': (3*(num_epochs/6) , 4*((num_epochs)/6) ),
                    'lev3': ( 5*((num_epochs)/6) , (num_epochs) )}

epochs_by_condition_testing = {}
selected_epochs_testing = []

# Iterate over condition ranges and select epochs for each condition
for condition, (start_epoch, end_epoch) in condition_ranges_testing.items():
    start_epoch = int(start_epoch)
    end_epoch = int(end_epoch)
    condition_epochs = testing_epochs[start_epoch:end_epoch].copy()
    epochs_by_condition_testing[condition] = condition_epochs
    selected_epochs_testing.append(condition_epochs)

# Concatenate epochs from different conditions into a single Epochs object
selected_epochs_testing = mne.concatenate_epochs(selected_epochs_testing)

# Print the number of epochs for each condition
for condition, epoch_data in epochs_by_condition_testing.items():
    num_epochs_condition = len(epoch_data)
    print(f"Number of epochs for {condition}: {num_epochs_condition}")

# Print the total number of selected epochs
print(f"Total number of selected epochs for testing: {len(selected_epochs_testing)}")


Not setting metadata
970 matching events found
No baseline correction applied
Number of epochs for lev1: 323
Number of epochs for lev2: 323
Number of epochs for lev3: 324
Total number of selected epochs for testing: 970


In [95]:
import pandas as pd
import numpy as np
from scipy.signal import welch



def calculate_psd_features(epoch_data, sfreq):
    psd_frequencies, psd_values = welch(epoch_data, fs=sfreq)
    psd_means = np.mean(psd_values, axis=1)  # Calculate mean PSD values for each channel
    psd_variances = np.var(psd_values, axis=1)  # Calculate variance of PSD values for each channel
    psd_std_devs = np.std(psd_values, axis=1)  # Calculate standard deviation of PSD values for each channel

    freq_min = 14
    freq_max = 30

    # Find the indices corresponding to the frequency range of interest
    idx_min = np.argmax(psd_frequencies > freq_min) - 1
    idx_max = np.argmax(psd_frequencies > freq_max)

    # Extract the PSD within the specified frequency range
    psd_values_alpha_band = psd_values[:, idx_min:idx_max]
    psd_means_alpha_band = np.mean(psd_values_alpha_band, axis=1)  # Calculate mean PSD values in alpha band for each channel
    psd_variances_alpha_band = np.var(psd_values_alpha_band, axis=1)  # Calculate variance of PSD values in alpha band for each channel
    psd_std_devs_alpha_band = np.std(psd_values_alpha_band, axis=1)  # Calculate standard deviation of PSD values in alpha band for each channel
    return psd_means_alpha_band, psd_variances_alpha_band, psd_std_devs_alpha_band

channel_names = epochs.ch_names
sfreq = epochs.info['sfreq']  # Sampling frequency

columns = [f'{feature}_{channel}' for feature in ['mean_psd', 'variance_psd', 'standarddev_psd'] for channel in channel_names]
columns.append('condition')
feature_df = pd.DataFrame(columns=columns)

def append_epoch_data(epoch_array, condition_label):
    global feature_df
    for epoch in epoch_array:
        psd_means, psd_variances, psd_std_devs = calculate_psd_features(epoch, sfreq)
        epoch_features = np.concatenate([psd_means, psd_variances, psd_std_devs])
        epoch_features = np.append(epoch_features, condition_label)
        epoch_df = pd.DataFrame([epoch_features], columns=columns)
        feature_df = pd.concat([feature_df, epoch_df], ignore_index=True)

append_epoch_data(epochs_by_condition['lev1'], 'lev1')
append_epoch_data(epochs_by_condition['lev2'], 'lev2')
append_epoch_data(epochs_by_condition['lev3'], 'lev3')

print("Shape of feature DataFrame:", feature_df.shape)
print(feature_df.head())


c:\Users\LEGION\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\signal\_spectral_py.py:1999: UserWarning: nperseg = 256 is greater than input length  = 48, using nperseg = 48
  warnings.warn('nperseg = {0:d} is greater than input length '


Shape of feature DataFrame: (10270, 49)
         mean_psd_Fp1       mean_psd_Fp2        mean_psd_C3  \
0   6602.058481791366  8737.537872642113  4394.761322012592   
1   5030.783753265276  6076.417358460207  4328.088509002579   
2   4048.688556190618  5807.189781917836   3935.19975583434   
3  3126.5628918802227  4363.879660877097  2540.463780311565   
4  4466.2332007763935  5833.289285673364  3731.085023576094   

          mean_psd_C4         mean_psd_P7        mean_psd_P8  \
0  3319.2729607885008   7209.508402203143  4505.945813228308   
1  2227.8797992242526   5791.111569183826  4438.134908080468   
2   2164.844059430652  4527.9652036440975  3810.500268982624   
3   1448.586714749527   3029.235073665243  2707.386303351086   
4   2309.861264741799  4092.6814984051884  4171.581431632499   

          mean_psd_O1         mean_psd_O2         mean_psd_F7  \
0   9113.881967119425  3653.8558509266063  3328.8532009942587   
1   6691.351677474442   4523.529380173445   3004.802111524347   
2

In [96]:
# Rename columns using a dictionary
new_column_names = {
    'mean_psd_Fp1': 'mean_psd_Fp1',
    'mean_psd_Fp2': 'mean_psd_Fp2',
    'mean_psd_C3': 'mean_psd_C3',
    'mean_psd_C4': 'mean_psd_C4',
    'mean_psd_P7': 'mean_psd_P7',
    'mean_psd_P8': 'mean_psd_P8',
    'mean_psd_O1': 'mean_psd_O1',
    'mean_psd_O2': 'mean_psd_O2',
    'mean_psd_F7': 'mean_psd_F7',
    'mean_psd_F8': 'mean_psd_F8',
    'standarddev_psd_Fp1': 'stddev_psd_Fp1',
    'standarddev_psd_Fp2': 'stddev_psd_Fp2',
    'standarddev_psd_C3': 'stddev_psd_C3',
    'standarddev_psd_C4': 'stddev_psd_C4',
    'standarddev_psd_P7': 'stddev_psd_P7',
    'standarddev_psd_P8': 'stddev_psd_P8',
    'standarddev_psd_O1': 'stddev_psd_O1',
    'standarddev_psd_O2': 'stddev_psd_O2',
    'standarddev_psd_F7': 'stddev_psd_F7',
    'standarddev_psd_F8': 'stddev_psd_F8'
}

# Rename columns using the rename() method
feature_df.rename(columns=new_column_names, inplace=True)

# Print the DataFrame
print(feature_df.head())


         mean_psd_Fp1       mean_psd_Fp2        mean_psd_C3  \
0   6602.058481791366  8737.537872642113  4394.761322012592   
1   5030.783753265276  6076.417358460207  4328.088509002579   
2   4048.688556190618  5807.189781917836   3935.19975583434   
3  3126.5628918802227  4363.879660877097  2540.463780311565   
4  4466.2332007763935  5833.289285673364  3731.085023576094   

          mean_psd_C4         mean_psd_P7        mean_psd_P8  \
0  3319.2729607885008   7209.508402203143  4505.945813228308   
1  2227.8797992242526   5791.111569183826  4438.134908080468   
2   2164.844059430652  4527.9652036440975  3810.500268982624   
3   1448.586714749527   3029.235073665243  2707.386303351086   
4   2309.861264741799  4092.6814984051884  4171.581431632499   

          mean_psd_O1         mean_psd_O2         mean_psd_F7  \
0   9113.881967119425  3653.8558509266063  3328.8532009942587   
1   6691.351677474442   4523.529380173445   3004.802111524347   
2   6259.102766040746  3182.4593223877423

In [97]:
feature_df

,mean_psd_Fp1,mean_psd_Fp2,mean_psd_C3,mean_psd_C4,mean_psd_P7,mean_psd_P8,mean_psd_O1,mean_psd_O2,mean_psd_F7,mean_psd_F8,...,stddev_psd_O2,stddev_psd_F7,stddev_psd_F8,standarddev_psd_F3,standarddev_psd_F4,standarddev_psd_T7,standarddev_psd_T8,standarddev_psd_P3,standarddev_psd_P4,condition
0,6602.058481791366,8737.537872642113,4394.761322012592,3319.2729607885008,7209.508402203143,4505.945813228308,9113.881967119425,3653.8558509266063,3328.8532009942587,8832.512733044287,...,6096.972547774705,4168.42759151675,14150.840726023638,17841.086855934613,7716.1635267639995,17719.661337939157,294236.7366023921,18209.802083293263,110744.51717909313,lev1
1,5030.783753265276,6076.417358460207,4328.088509002579,2227.8797992242526,5791.111569183826,4438.134908080468,6691.351677474442,4523.529380173445,3004.802111524347,9166.844470609289,...,6979.448912824641,4429.5497737544865,14698.373195446602,16357.156940212095,3029.235511995939,15872.489471796895,37410.56820880773,16525.64750827445,20931.449168340274,lev1
2,4048.688556190618,5807.189781917836,3935.19975583434,2164.844059430652,4527.9652036440975,3810.500268982624,6259.102766040746,3182.4593223877423,3048.7593624800393,5958.691332838877,...,4820.656746255473,3917.2295003022914,9471.886663881922,11884.38680931293,3042.3369263253153,11790.902038772796,3264.3974232269265,12481.189024434334,3107.86090521498,lev1
3,3126.5628918802227,4363.879660877097,2540.463780311565,1448.586714749527,3029.235073665243,2707.386303351086,4094.4091859946066,2284.95188209817,1921.7027029915218,5718.368303476286,...,3514.035109969654,2385.414433036654,9077.02101292832,11473.812293332163,1570.459770621817,10936.22144418774,1574.759691346513,11365.436680414208,1660.2915879668622,lev1
4,4466.2332007763935,5833.289285673364,3731.085023576094,2309.861264741799,4092.6814984051884,4171.581431632499,4277.330840814739,3908.6223943028226,2968.121520222362,7568.193012989672,...,5493.764560720933,3369.905128294881,11069.23946185855,12467.476417582326,1341.0590921625374,13294.36626974968,3710.8133105057796,13560.073226511611,1877.2415782545972,lev1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10265,2354.629077505681,2299.2150058831257,2372.32941120333,2323.1102444906974,2223.285680149836,4049.396445178396,774.662659754526,2522.39695535604,5466.540032690443,5401.767362416383,...,2506.741990280256,5746.687577676446,5679.188220330877,5745.334280647424,5596.2505120380665,3084.231142002481,5799.150024502735,2728.653107402629,6291.762310204871,lev3
10266,2926.3937409191294,2873.201250515049,2900.8988891771396,2821.8663879638743,2767.9240967169508,3699.4526301253622,1044.5434096627687,2848.8669497077894,2790.39284474915,2822.4083227373203,...,1757.1904189796285,2164.1782006065564,2213.3765189506316,2181.3549866355447,2206.5421607238272,6457.714462756516,2234.1609459290667,2357.095715643188,5266.462309859479,lev3
10267,3060.442603126936,3083.8623982838794,3053.500565560643,3069.70910202119,3118.860639904118,4122.401775391818,1725.3392141494971,3175.071954850836,5018.9695479049315,5169.3235405353,...,3285.039306687031,6363.598993244584,6581.175102363817,6496.096486100471,6466.018630621367,7228.191407446372,6063.401012567815,4447.426877972059,13177.365589032002,lev3
10268,4919.603311006676,4839.844370692213,4913.754296888276,4901.556624235064,4736.999156766041,7131.855272287167,4642.932163260099,4467.811126229766,6924.676841986724,6866.845878857741,...,2758.011388339327,4902.734934300516,4901.9033899798105,4832.620342728205,4899.752259113823,9825.065836018357,4857.908622507901,6517.421085097856,8723.4918245399,lev3


In [98]:
# Create a DataFrame to store the features extracted from testing epochs
feature_columns = [f'{feature}_{channel}' for feature in ['mean_psd', 'variance_psd', 'standarddev_psd'] for channel in ch_names]
feature_columns.append('condition')  # Add a column for condition labels
testing_feature_df = pd.DataFrame(columns=feature_columns)

# Function to calculate PSD features for testing epochs
def append_testing_epoch_data(epoch_array, condition_label):
    global testing_feature_df
    for epoch in epoch_array:
        psd_means, psd_variances, psd_std_devs = calculate_psd_features(epoch, sfreq)
        epoch_features = np.concatenate([psd_means, psd_variances, psd_std_devs])
        epoch_features = np.append(epoch_features, condition_label)
        epoch_df = pd.DataFrame([epoch_features], columns=feature_columns)
        testing_feature_df = pd.concat([testing_feature_df, epoch_df], ignore_index=True)

# Apply the feature extraction function to testing epochs for each condition
append_testing_epoch_data(epochs_by_condition_testing['lev1'], 'lev1')
append_testing_epoch_data(epochs_by_condition_testing['lev2'], 'lev2')
append_testing_epoch_data(epochs_by_condition_testing['lev3'], 'lev3')

# Print the shape and first few rows of the testing feature DataFrame
print("Shape of testing feature DataFrame:", testing_feature_df.shape)
print(testing_feature_df.head())


c:\Users\LEGION\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\signal\_spectral_py.py:1999: UserWarning: nperseg = 256 is greater than input length  = 48, using nperseg = 48
  warnings.warn('nperseg = {0:d} is greater than input length '


Shape of testing feature DataFrame: (970, 49)
           mean_psd_Fp1          mean_psd_Fp2           mean_psd_C3  \
0     5.995051558572906    5.9950533236639885     5.995052492679263   
1     0.610405541439272    0.6104056786651328    0.6104056543459885   
2  0.030426515809005194  0.030426486826389633  0.030426500247449972   
3   0.01832654946759057   0.01832653278847269  0.018326540853469214   
4   0.01279074827379908   0.01279075862218912  0.012790752810703687   

            mean_psd_C4           mean_psd_P7           mean_psd_P8  \
0     5.995054941485934     5.995048529705496     5.995050985476627   
1    0.6104059867439162    0.6104055046747813    0.6104054315129362   
2   0.03042650741298379  0.030426526528500544  0.030426486882650417   
3  0.018326539050285532  0.018326540359750565  0.018326523749843872   
4  0.012790754409555759  0.012790751562216053  0.012790744387648269   

            mean_psd_O1           mean_psd_O2           mean_psd_F7  \
0      5.99505126567832     5

In [99]:
new_column_names = {
    'mean_psd_Fp1': 'mean_psd_Fp1',
    'mean_psd_Fp2': 'mean_psd_Fp2',
    'mean_psd_C3': 'mean_psd_C3',
    'mean_psd_C4': 'mean_psd_C4',
    'mean_psd_P7': 'mean_psd_P7',
    'mean_psd_P8': 'mean_psd_P8',
    'mean_psd_O1': 'mean_psd_O1',
    'mean_psd_O2': 'mean_psd_O2',
    'mean_psd_F7': 'mean_psd_F7',
    'mean_psd_F8': 'mean_psd_F8',
    'standarddev_psd_Fp1': 'stddev_psd_Fp1',
    'standarddev_psd_Fp2': 'stddev_psd_Fp2',
    'standarddev_psd_C3': 'stddev_psd_C3',
    'standarddev_psd_C4': 'stddev_psd_C4',
    'standarddev_psd_P7': 'stddev_psd_P7',
    'standarddev_psd_P8': 'stddev_psd_P8',
    'standarddev_psd_O1': 'stddev_psd_O1',
    'standarddev_psd_O2': 'stddev_psd_O2',
    'standarddev_psd_F7': 'stddev_psd_F7',
    'standarddev_psd_F8': 'stddev_psd_F8'
}

testing_feature_df.rename(columns=new_column_names, inplace=True)
print(testing_feature_df.head())


           mean_psd_Fp1          mean_psd_Fp2           mean_psd_C3  \
0     5.995051558572906    5.9950533236639885     5.995052492679263   
1     0.610405541439272    0.6104056786651328    0.6104056543459885   
2  0.030426515809005194  0.030426486826389633  0.030426500247449972   
3   0.01832654946759057   0.01832653278847269  0.018326540853469214   
4   0.01279074827379908   0.01279075862218912  0.012790752810703687   

            mean_psd_C4           mean_psd_P7           mean_psd_P8  \
0     5.995054941485934     5.995048529705496     5.995050985476627   
1    0.6104059867439162    0.6104055046747813    0.6104054315129362   
2   0.03042650741298379  0.030426526528500544  0.030426486882650417   
3  0.018326539050285532  0.018326540359750565  0.018326523749843872   
4  0.012790754409555759  0.012790751562216053  0.012790744387648269   

            mean_psd_O1           mean_psd_O2           mean_psd_F7  \
0      5.99505126567832     5.995048545037344     5.994993290028031   
1   

In [100]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X = feature_df.drop(columns=['condition'])
y = feature_df['condition']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

y_pred = rf_classifier.predict(X_test)

# Prepare feature matrix X and target vector y for the testing data
X_testing = testing_feature_df.drop(columns=['condition'])
y_testing = testing_feature_df['condition']

# Making predictions on the testing data and calculating accuracy
y_pred_testing = rf_classifier.predict(X_testing)
accuracy_testing = accuracy_score(y_testing, y_testing)
print("Accuracy on testing data:", accuracy_testing)


Accuracy on testing data: 1.0


In [101]:
# def append_epoch_data(epoch_array, condition_label):
#         global feature_df
#         for epoch in epoch_array:
#             psd_means, psd_variances, psd_std_devs = calculate_psd_features(epoch, sfreq)
#             epoch_features = np.concatenate([psd_means, psd_variances, psd_std_devs])
#             epoch_features = np.append(epoch_features, condition_label)
#             epoch_df = pd.DataFrame([epoch_features], columns=columns)
#             feature_df = pd.concat([feature_df, epoch_df], ignore_index=True)

# def calculate_psd_features(epoch_data, sfreq):
#         psd_frequencies, psd_values = welch(epoch_data, fs=sfreq)
#         psd_means = np.mean(psd_values, axis=1)  # Calculate mean PSD values for each channel
#         psd_variances = np.var(psd_values, axis=1)  # Calculate variance of PSD values for each channel
#         psd_std_devs = np.std(psd_values, axis=1)  # Calculate standard deviation of PSD values for each channel

#         freq_min = 14
#         freq_max = 30

#         # Find the indices corresponding to the frequency range of interest
#         idx_min = np.argmax(psd_frequencies > freq_min) - 1
#         idx_max = np.argmax(psd_frequencies > freq_max)

#         # Extract the PSD within the specified frequency range
#         psd_values_alpha_band = psd_values[:, idx_min:idx_max]
#         psd_means_alpha_band = np.mean(psd_values_alpha_band, axis=1)  # Calculate mean PSD values in alpha band for each channel
#         psd_variances_alpha_band = np.var(psd_values_alpha_band, axis=1)  # Calculate variance of PSD values in alpha band for each channel
#         psd_std_devs_alpha_band = np.std(psd_values_alpha_band, axis=1)  # Calculate standard deviation of PSD values in alpha band for each channel
#         return psd_means_alpha_band, psd_variances_alpha_band, psd_std_devs_alpha_band


In [102]:
# import matplotlib.pyplot as plt

# # Initialize lists to store durations and accuracies
# durations = []
# accuracies = []

# # Define a range of durations to try
# duration_range = [0.1, 0.2, 0.3, 0.4, 0.5,0.6,0.7,0.8,0.9,1.0]

# for duration in duration_range:
#     epochs = mne.make_fixed_length_epochs(raw, duration=duration, overlap=0, preload=True)
    
#     num_epochs = len(epochs)

#     eeg_data = epochs.get_data()

#     # condition_ranges = {'lev1': (0, (num_epochs/3)),
#     #                     'lev2': ((num_epochs/3) , ((2*num_epochs)/3) ),
#     #                     'lev3': ((2*num_epochs)/3 , (num_epochs) )}
#     condition_ranges = {'lev1': (0, (num_epochs/6)),
#                     'lev2': (3*(num_epochs/6) , 4*((num_epochs)/6) ),
#                     'lev3': ( 5*((num_epochs)/6) , (num_epochs) )}

#     epochs_by_condition = {}
#     selected_epochs = []

#     for condition, (start_epoch, end_epoch) in condition_ranges.items():
#         start_epoch = int(start_epoch)
#         end_epoch = int(end_epoch)
#         condition_epochs = epochs[start_epoch:end_epoch+1].copy()
#         epochs_by_condition[condition] = condition_epochs
#         selected_epochs.append(condition_epochs)

#     selected_epochs = mne.concatenate_epochs(selected_epochs)
#     import pandas as pd
#     import numpy as np
#     from scipy.signal import welch



    
#     channel_names = epochs.ch_names
#     sfreq = epochs.info['sfreq']  # Sampling frequency

#     columns = [f'{feature}_{channel}' for feature in ['mean_psd', 'variance_psd', 'standarddev_psd'] for channel in channel_names]
#     columns.append('condition')
#     feature_df = pd.DataFrame(columns=columns)

    

#     append_epoch_data(epochs_by_condition['lev1'], 'lev1')
#     append_epoch_data(epochs_by_condition['lev2'], 'lev2')
#     append_epoch_data(epochs_by_condition['lev3'], 'lev3')
#     # Rename columns using a dictionary
#     new_column_names = {
#         'mean_psd_Fp1': 'mean_psd_Fp1',
#         'mean_psd_Fp2': 'mean_psd_Fp2',
#         'mean_psd_C3': 'mean_psd_C3',
#         'mean_psd_C4': 'mean_psd_C4',
#         'mean_psd_P7': 'mean_psd_P7',
#         'mean_psd_P8': 'mean_psd_P8',
#         'mean_psd_O1': 'mean_psd_O1',
#         'mean_psd_O2': 'mean_psd_O2',
#         'mean_psd_F7': 'mean_psd_F7',
#         'mean_psd_F8': 'mean_psd_F8',
#         'standarddev_psd_Fp1': 'stddev_psd_Fp1',
#         'standarddev_psd_Fp2': 'stddev_psd_Fp2',
#         'standarddev_psd_C3': 'stddev_psd_C3',
#         'standarddev_psd_C4': 'stddev_psd_C4',
#         'standarddev_psd_P7': 'stddev_psd_P7',
#         'standarddev_psd_P8': 'stddev_psd_P8',
#         'standarddev_psd_O1': 'stddev_psd_O1',
#         'standarddev_psd_O2': 'stddev_psd_O2',
#         'standarddev_psd_F7': 'stddev_psd_F7',
#         'standarddev_psd_F8': 'stddev_psd_F8'
#     }

#     # Rename columns using the rename() method
#     feature_df.rename(columns=new_column_names, inplace=True)
#     # Ensure the rest of your code is within this loop so that it executes for each duration
    
#     # Train the classifier and calculate accuracy
#     X = feature_df.drop(columns=['condition'])
#     y = feature_df['condition']
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#     rf_classifier.fit(X_train, y_train)
#     y_pred = rf_classifier.predict(X_test)
#     accuracy = accuracy_score(y_test, y_pred)
    
#     # Append duration and accuracy to lists
#     durations.append(duration)
#     accuracies.append(accuracy)

# # Plot duration vs accuracy
# plt.plot(durations, accuracies, marker='o')
# plt.xlabel('Duration')
# plt.ylabel('Accuracy')
# plt.title('Duration vs Accuracy')
# plt.grid(True)
# plt.show()


In [103]:
# import matplotlib.pyplot as plt

# # Initialize lists to store durations and accuracies
# durations = []
# accuracies = []

# # Define a range of durations to try
# duration_range = [0.05,0.1, 0.2, 0.3, 0.4, 0.5,0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.3,1.4,1.5,2,3]

# for duration in duration_range:
#     epochs = mne.make_fixed_length_epochs(raw, duration=duration, overlap=0, preload=True)
    
#     num_epochs = len(epochs)

#     eeg_data = epochs.get_data()

#     # condition_ranges = {'lev1': (0, (num_epochs/3)),
#     #                     'lev2': ((num_epochs/3) , ((2*num_epochs)/3) ),
#     #                     'lev3': ((2*num_epochs)/3 , (num_epochs) )}
#     condition_ranges = {'lev1': (0, (num_epochs/6)),
#                     'lev2': (3*(num_epochs/6) , 4*((num_epochs)/6) ),
#                     'lev3': ( 5*((num_epochs)/6) , (num_epochs) )}

#     epochs_by_condition = {}
#     selected_epochs = []

#     for condition, (start_epoch, end_epoch) in condition_ranges.items():
#         start_epoch = int(start_epoch)
#         end_epoch = int(end_epoch)
#         condition_epochs = epochs[start_epoch:end_epoch+1].copy()
#         epochs_by_condition[condition] = condition_epochs
#         selected_epochs.append(condition_epochs)

#     selected_epochs = mne.concatenate_epochs(selected_epochs)
#     import pandas as pd
#     import numpy as np
#     from scipy.signal import welch



    
#     channel_names = epochs.ch_names
#     sfreq = epochs.info['sfreq']  # Sampling frequency

#     columns = [f'{feature}_{channel}' for feature in ['mean_psd', 'variance_psd', 'standarddev_psd'] for channel in channel_names]
#     columns.append('condition')
#     feature_df = pd.DataFrame(columns=columns)

    

#     append_epoch_data(epochs_by_condition['lev1'], 'lev1')
#     append_epoch_data(epochs_by_condition['lev2'], 'lev2')
#     append_epoch_data(epochs_by_condition['lev3'], 'lev3')
#     # Rename columns using a dictionary
#     new_column_names = {
#         'mean_psd_Fp1': 'mean_psd_Fp1',
#         'mean_psd_Fp2': 'mean_psd_Fp2',
#         'mean_psd_C3': 'mean_psd_C3',
#         'mean_psd_C4': 'mean_psd_C4',
#         'mean_psd_P7': 'mean_psd_P7',
#         'mean_psd_P8': 'mean_psd_P8',
#         'mean_psd_O1': 'mean_psd_O1',
#         'mean_psd_O2': 'mean_psd_O2',
#         'mean_psd_F7': 'mean_psd_F7',
#         'mean_psd_F8': 'mean_psd_F8',
#         'standarddev_psd_Fp1': 'stddev_psd_Fp1',
#         'standarddev_psd_Fp2': 'stddev_psd_Fp2',
#         'standarddev_psd_C3': 'stddev_psd_C3',
#         'standarddev_psd_C4': 'stddev_psd_C4',
#         'standarddev_psd_P7': 'stddev_psd_P7',
#         'standarddev_psd_P8': 'stddev_psd_P8',
#         'standarddev_psd_O1': 'stddev_psd_O1',
#         'standarddev_psd_O2': 'stddev_psd_O2',
#         'standarddev_psd_F7': 'stddev_psd_F7',
#         'standarddev_psd_F8': 'stddev_psd_F8'
#     }

#     # Rename columns using the rename() method
#     feature_df.rename(columns=new_column_names, inplace=True)
#     # Ensure the rest of your code is within this loop so that it executes for each duration
    
#     # Train the classifier and calculate accuracy
#     X = feature_df.drop(columns=['condition'])
#     y = feature_df['condition']
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    

#     # Initialize the Gradient Boosting classifier
#     gb_classifier = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)

#     # Train the classifier on the training data
#     gb_classifier.fit(X_train, y_train)

#     # Predict the labels for the test set
#     y_pred = gb_classifier.predict(X_test)

#     # Calculate the accuracy of the model
#     accuracy = accuracy_score(y_test, y_pred)
    
#     # Append duration and accuracy to lists
#     durations.append(duration)
#     accuracies.append(accuracy)

# # Plot duration vs accuracy
# plt.plot(durations, accuracies, marker='o')
# plt.xlabel('Duration')
# plt.ylabel('Accuracy')
# plt.title('Duration vs Accuracy')
# plt.grid(True)
# plt.show()
